In [1]:
import subprocess
import os
import sys

def run_shell_command(command):
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        # Print output in real-time
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                print(output.strip())
                
        # Get the return code
        return_code = process.poll()
        
        # Print any errors if the command failed
        if return_code != 0:
            print("Error output:", file=sys.stderr)
            print(process.stderr.read(), file=sys.stderr)
            raise subprocess.CalledProcessError(return_code, command)
            
    except Exception as e:
        print(f"Error executing command: {e}", file=sys.stderr)
        raise
os.environ["JAVA_HOME"] = "/Users/pnilangekar/.jenv/versions/21"
print(f"Set JAVA_HOME to: {os.environ['JAVA_HOME']}")
os.environ['SPARK_VERSION'] = '3.5.6'  # Set Spark version
print(f"SPARK_VERSION set to: {os.environ['SPARK_VERSION']}")
# Execute the setup_hms.sh script with clean_metastore argument
run_shell_command("./setup_hms.sh clean_metastore")


Set JAVA_HOME to: /Users/pnilangekar/.jenv/versions/21
SPARK_VERSION set to: 3.5.6
Hive distro at /Users/pnilangekar/apache-hive-3.1.3-bin
Vefitying Spark conf...
# HIVE_METASTORE_ICEBERG_TESTCONF
Hive metastore iceberg conf already set
Killing any running metastore server...
Deleting metastore database and warehouse...
Initializing metastore schema...
Metastore connection URL:	 jdbc:derby:;databaseName=/tmp/data/hms/metastore_db;;create=true
Metastore Connection Driver :	 org.apache.derby.jdbc.EmbeddedDriver
Metastore connection User:	 APP
Starting metastore schema initialization to 3.1.0
Initialization script hive-schema-3.1.0.derby.sql
Initialization script completed
schemaTool completed
Starting metastore server...


In [2]:
import os
from pyspark.sql import SparkSession


# Set JAVA_HOME directly (same value that works in your shell)
os.environ["JAVA_HOME"] = "/Users/pnilangekar/.jenv/versions/21"
print(f"Set JAVA_HOME to: {os.environ['JAVA_HOME']}")

os.environ["SPARK_HOME"] ="/Users/pnilangekar/spark-3.5.5-bin-hadoop3"
print("Spark home: ", os.environ.get("SPARK_HOME"))
print("JAVA_HOME: ", os.environ.get("JAVA_HOME"))


iceberg_version = "1.9.0"
spark_version = "3.5"
scala_version = "2.12"
iceberg_package = f"org.apache.iceberg:iceberg-spark-runtime-{spark_version}_{scala_version}:{iceberg_version}"

# --- Spark Session Builder ---
# This configuration is cleaner and follows best practices.
spark = SparkSession.builder \
    .appName("Hive Federation Demo") \
    .config("spark.jars.packages", iceberg_package) \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.spark_catalog.uri", "thrift://localhost:9083") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "/tmp/data/spark-warehouse") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

# --- Verification ---
# Print the Spark version and some key configurations to verify.
print(f"Spark Session created successfully with Spark version: {spark.version}")
conf = spark.sparkContext.getConf()
print(f"Spark Configuration: {conf.getAll()}")


# --- Example Usage: Create Namespaces and Tables ---
# The key fix is adding "USING iceberg" to your CREATE TABLE statements.
# Create namespaces and tables
commands = [
    "CREATE NAMESPACE IF NOT EXISTS ns1",
    "CREATE NAMESPACE IF NOT EXISTS ns2",
    "DROP TABLE IF EXISTS ns1.table1",
    "DROP TABLE IF EXISTS ns2.table2",
    "CREATE TABLE ns1.table1 (key STRING, value STRING, version INT) STORED AS ICEBERG",
    "CREATE TABLE ns2.table2 (key STRING, value STRING, version INT) STORED AS ICEBERG"
]

# Execute each command and print the result
for cmd in commands:
    print(f"\nExecuting: {cmd}")
    try:
        result = spark.sql(cmd)
        if cmd.strip().upper().startswith(("SHOW", "SELECT", "DESCRIBE")):
            result.show()
        else:
            print("Success!")
    except Exception as e:
        print(f"Error: {str(e)}")

# Insert some data
print("\n--- Inserting Data ---")
spark.sql("INSERT INTO spark_catalog.ns1.table1 VALUES ('Engine', 'Spark', 1), ('Engine', 'Apache Spark', 2), ('Catalog', 'Hive', 1)")
spark.sql("INSERT INTO spark_catalog.ns2.table2 VALUES ('Engine', 'Snowflake', 1), ('Alt Engine', 'Apache Spark', 1), ('Catalog', 'Open Catalog', 1)")
print("Data inserted successfully.")

# Verify the tables were created
print("\n--- Verifying Tables ---")
print("Listing all tables in ns1:")
spark.sql("SHOW TABLES IN spark_catalog.ns1").show()
print("\nListing all tables in ns2:")
spark.sql("SHOW TABLES IN spark_catalog.ns2").show()

# Verify data can be read
print("\n--- Reading Data ---")
print("Data from ns1.table1:")
spark.table("spark_catalog.ns1.table1").show()
print("\nData from ns2.table2:")
spark.table("spark_catalog.ns2.table2").show()


# Stop the session when done
spark.stop()
print("\nSpark session stopped.")

Set JAVA_HOME to: /Users/pnilangekar/.jenv/versions/21
Spark home:  /Users/pnilangekar/spark-3.5.5-bin-hadoop3
JAVA_HOME:  /Users/pnilangekar/.jenv/versions/21
:: loading settings :: url = jar:file:/Users/pnilangekar/spark-3.5.5-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/pnilangekar/.ivy2/cache
The jars for the packages stored in: /Users/pnilangekar/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e91d993f-d90e-46d6-9069-ab828ec7d66c;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.0 in central
:: resolution report :: resolve 52ms :: artifacts dl 1ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: or

Spark Session created successfully with Spark version: 3.5.5
Spark Configuration: [('spark.files', 'file:///Users/pnilangekar/.ivy2/jars/org.apache.iceberg_iceberg-spark-runtime-3.5_2.12-1.9.0.jar'), ('spark.jars', 'file:///Users/pnilangekar/.ivy2/jars/org.apache.iceberg_iceberg-spark-runtime-3.5_2.12-1.9.0.jar'), ('spark.sql.variable.substitute', 'true'), ('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkCatalog'), ('spark.sql.catalog.hadoop_cat.warehouse', 'file:///tmp/warehouse'), ('spark.sql.defaultCatalog', 'spark_catalog'), ('spark.app.id', 'local-1755217205750'), ('spark.sql.warehouse.dir', '/tmp/data/spark-warehouse'), ('spark.serializer.objectStreamReset', '100'), ('spark.app.name', 'Hive Federation Demo'), ('spark.master', 'local[*]'), ('spark.sql.catalog.spark_catalog.uri', 'thrift://localhost:9083'), ('spark.submit.deployMode', 'client'), ('spark.app.initial.jar.urls', 'spark://127.0.0.1:61451/jars/org.apache.iceberg_iceberg-spark-runtime-3.5_2.12-1.9.0.jar

In [4]:
# Set up POLARIS auth token. 
import os
import requests
import json

POLARIS_HOST = os.getenv('POLARIS_HOST', 'localhost')
BASE_URL = f"http://{POLARIS_HOST}:8181"

def get_polaris_token():
    response = requests.post(f"{BASE_URL}/api/catalog/v1/oauth/tokens",
                             data={'grant_type': 'client_credentials', 'scope': 'PRINCIPAL_ROLE:ALL'},
                             auth=('root', 's3cr3t'))
    return response.json()['access_token']

POLARIS_TOKEN = get_polaris_token()

os.environ['POLARIS_TOKEN'] = POLARIS_TOKEN

In [5]:
import subprocess

def curl_command(cmd):
    process = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    print(f"Status Code: {process.returncode}")
    print(process.stdout if process.stdout else process.stderr)

In [6]:
# Define the JSON payload
json_data = '''{
  "type": "EXTERNAL",
  "name": "federated_hive",
  "connectionConfigInfo": {
    "connectionType": "HIVE",
    "uri": "thrift://127.0.0.1:9083",
    "warehouse": "hms",
    "authenticationParameters": {
      "authenticationType": "IMPLICIT"
    }
  },
  "properties": {
    "default-base-location": "file:///tmp/data/spark-warehouse"
  },
  "storageConfigInfo": {
    "storageType": "FILE",
    "allowedLocations": [
      "file:///tmp/data/spark-warehouse"
    ]
  }
}'''

# Construct and execute the curl command
cmd = f'''curl -X POST http://polaris:8181/api/management/v1/catalogs \
    --resolve polaris:8181:127.0.0.1 \
    -H "Accepts: application/json" \
    -H "Content-Type: application/json" \
    -H "Authorization: Bearer $POLARIS_TOKEN" \
    -d '{json_data}' '''

curl_command(cmd)

Status Code: 0
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   473    0     0  100   473      0   4106 --:--:-- --:--:-- --:--:--  4113



In [7]:
cmd = f'''curl -i -X GET http://localhost:8181/api/management/v1/catalogs/federated_hive/ \
  -H "Authorization: Bearer $POLARIS_TOKEN" -H "Accept: application/json" -H "Content-Type: application/json" '''

curl_command(cmd)

Status Code: 0
HTTP/1.1 200 OK
Content-Type: application/json;charset=UTF-8
content-length: 462

{"connectionConfigInfo":{"warehouse":"hms","connectionType":"HIVE","uri":"thrift://127.0.0.1:9083","authenticationParameters":{"authenticationType":"IMPLICIT"}},"type":"EXTERNAL","name":"federated_hive","properties":{"default-base-location":"file:///tmp/data/spark-warehouse"},"createTimestamp":1755217258155,"lastUpdateTimestamp":1755217258155,"entityVersion":1,"storageConfigInfo":{"storageType":"FILE","allowedLocations":["file:///tmp/data/spark-warehouse"]}}


In [8]:
# List all tables in the ns1 namespace. 
cmd = f'''curl -i -X GET http://localhost:8181/api/catalog/v1/federated_hive/namespaces/ns1/tables/ \
  -H "Authorization: Bearer $POLARIS_TOKEN" -H "Accept: application/json" -H "Content-Type: application/json" '''

curl_command(cmd)

Status Code: 0
HTTP/1.1 200 OK
Content-Type: application/json;charset=UTF-8
content-length: 78

{"identifiers":[{"namespace":["ns1"],"name":"table1"}],"next-page-token":null}


In [9]:
# List all namespaces. 
cmd = f'''curl -i -X GET http://localhost:8181/api/catalog/v1/federated_hive/namespaces/ \
  -H "Authorization: Bearer $POLARIS_TOKEN" -H "Accept: application/json" -H "Content-Type: application/json" '''

curl_command(cmd)


Status Code: 0
HTTP/1.1 200 OK
Content-Type: application/json;charset=UTF-8
content-length: 67

{"namespaces":[["default"],["ns1"],["ns2"]],"next-page-token":null}


In [10]:
# List all tables in the ns1 namespace. 
cmd = f'''curl -i -X GET http://localhost:8181/api/catalog/v1/federated_hive/namespaces/ns1/tables/ \
  -H "Authorization: Bearer $POLARIS_TOKEN" -H "Accept: application/json" -H "Content-Type: application/json" '''

curl_command(cmd)

Status Code: 0
HTTP/1.1 200 OK
Content-Type: application/json;charset=UTF-8
content-length: 78

{"identifiers":[{"namespace":["ns1"],"name":"table1"}],"next-page-token":null}


In [11]:
# List table metadata for ns1.table1. 
cmd = f'''curl -i -X GET http://localhost:8181/api/catalog/v1/federated_hive/namespaces/ns1/tables/table1 \
  -H "Authorization: Bearer $POLARIS_TOKEN" -H "Accept: application/json" -H "Content-Type: application/json" '''

curl_command(cmd)

Status Code: 0
HTTP/1.1 200 OK
Content-Type: application/json;charset=UTF-8
content-length: 2022
ETag: W/"2dcfc0ca37b4374c31d5ec504a09f0965c0227ed84aa7f33433684a301763ffd"

{"metadata-location":"file:/tmp/data/spark-warehouse/ns1.db/table1/metadata/00001-11ea7768-348d-449e-a316-f5fff8cc725e.metadata.json","metadata":{"format-version":2,"table-uuid":"6a6e2a44-2fc2-4e9a-a593-f65f7125e4fa","location":"file:/tmp/data/spark-warehouse/ns1.db/table1","last-sequence-number":1,"last-updated-ms":1755217209166,"last-column-id":3,"current-schema-id":0,"schemas":[{"type":"struct","schema-id":0,"fields":[{"id":1,"name":"key","required":false,"type":"string"},{"id":2,"name":"value","required":false,"type":"string"},{"id":3,"name":"version","required":false,"type":"int"}]}],"default-spec-id":0,"partition-specs":[{"spec-id":0,"fields":[]}],"last-partition-id":999,"default-sort-order-id":0,"sort-orders":[{"order-id":0,"fields":[]}],"properties":{"owner":"pnilangekar","hive.stored-as":"ICEBERG","write.pa

In [12]:
# Grant TABLE_WRITE_DATA to the catalog_admin role. 
json_data = '''{"type": "catalog", "privilege": "TABLE_WRITE_DATA"}'''
cmd = f'''curl -i -X PUT -H "Authorization: Bearer $POLARIS_TOKEN" -H "Accept: application/json" -H "Content-Type: application/json" \
    http://localhost:8181/api/management/v1/catalogs/federated_hive/catalog-roles/catalog_admin/grants \
  -d '{json_data}' '''

curl_command(cmd)

Status Code: 0
HTTP/1.1 201 Created
content-encoding: identity
content-length: 0




In [13]:
# Assign the catalog_admin role to the service_admin.
json_data = '''{"name": "catalog_admin"}'''
cmd = f'''curl -i -X PUT -H "Authorization: Bearer $POLARIS_TOKEN" -H "Accept: application/json" -H "Content-Type: application/json" \
    http://localhost:8181/api/management/v1/principal-roles/service_admin/catalog-roles/federated_hive \
  -d '{json_data}' '''

curl_command(cmd)

Status Code: 0
HTTP/1.1 201 Created
content-encoding: identity
content-length: 0




In [14]:
import os
from pyspark.sql import SparkSession


iceberg_version = "1.9.0"
spark_version = "3.5"
scala_version = "2.12"
iceberg_package = f"org.apache.iceberg:iceberg-spark-runtime-{spark_version}_{scala_version}:{iceberg_version}"

# Create Spark session with the same configurations as spark-sql command
spark = SparkSession.builder \
    .appName("Polaris Hive Federation") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.jars.packages",  "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.iceberg:iceberg-aws-bundle:1.7.1,software.amazon.awssdk:bundle:2.30.25,net.snowflake:snowflake-jdbc:3.13.32,com.azure:azure-storage-file-datalake:12.17.0,software.amazon.s3tables:s3-tables-catalog-for-iceberg-runtime:0.1.5") \
     .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.polaris.token", os.environ.get('POLARIS_TOKEN')) \
    .config("spark.sql.catalog.polaris.type", "rest") \
    .config("spark.sql.catalog.polaris.uri", "http://localhost:8181/api/catalog") \
    .config("spark.sql.catalog.polaris.warehouse", "federated_hive") \
    .config("spark.sql.defaultCatalog", "polaris") \
    .config("spark.sql.catalog.polaris.client.region", "us-east-1") \
    .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation", "vended-credentials") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

print("Spark version:", spark.version)

# Verify configuration
print("Active Spark Configuration:")
print(spark.sparkContext.getConf().getAll())

Spark version: 3.5.5
Active Spark Configuration:
[('spark.files', 'file:///Users/pnilangekar/.ivy2/jars/org.apache.iceberg_iceberg-spark-runtime-3.5_2.12-1.9.0.jar'), ('spark.jars', 'file:///Users/pnilangekar/.ivy2/jars/org.apache.iceberg_iceberg-spark-runtime-3.5_2.12-1.9.0.jar'), ('spark.sql.variable.substitute', 'true'), ('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkCatalog'), ('spark.sql.catalog.hadoop_cat.warehouse', 'file:///tmp/warehouse'), ('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation', 'vended-credentials'), ('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.iceberg:iceberg-aws-bundle:1.7.1,software.amazon.awssdk:bundle:2.30.25,net.snowflake:snowflake-jdbc:3.13.32,com.azure:azure-storage-file-datalake:12.17.0,software.amazon.s3tables:s3-tables-catalog-for-iceberg-runtime:0.1.5'), ('spark.sql.catalog.polaris.type', 'rest'), ('spark.sql.catalog.polaris.token', 'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJ

In [15]:
# List all namespaces
print("Listing all namespaces:")
spark.sql("SHOW NAMESPACES").show()

# List tables in ns1 and ns2
print("\nListing tables in ns1:")
spark.sql("SHOW TABLES IN ns1").show()

print("\nListing tables in ns2:")
spark.sql("SHOW TABLES IN ns2").show()

# Insert sample data into tables
insert_commands = [
    """INSERT INTO ns1.table1 (key, value, version) 
       VALUES 
       ('key1', 'value1', 1),
       ('key2', 'value2', 2),
       ('key3', 'value3', 3)""",
    
    """INSERT INTO ns2.table2 (key, value, version)
       VALUES
       ('keyA', 'valueA', 10),
       ('keyB', 'valueB', 20),
       ('keyC', 'valueC', 30)"""
]

# Execute inserts
for cmd in insert_commands:
    print(f"\nExecuting: {cmd}")
    spark.sql(cmd)
    print("Success!")

# Verify data in tables
print("\nData in ns1.table1:")
spark.sql("SELECT * FROM ns1.table1").show()

print("\nData in ns2.table2:")
spark.sql("SELECT * FROM ns2.table2").show()


Listing all namespaces:


25/08/14 17:21:19 WARN AuthManagers: Inferring rest.auth.type=oauth2 since property token was provided. Please explicitly set rest.auth.type to avoid this warning.
25/08/14 17:21:19 WARN OAuth2Manager: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to http://localhost:8181/api/catalog/v1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


+---------+
|namespace|
+---------+
|  default|
|      ns1|
|      ns2|
+---------+


Listing tables in ns1:
+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|      ns1|   table1|      false|
+---------+---------+-----------+


Listing tables in ns2:
+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|      ns2|   table2|      false|
+---------+---------+-----------+


Executing: INSERT INTO ns1.table1 (key, value, version) 
       VALUES 
       ('key1', 'value1', 1),
       ('key2', 'value2', 2),
       ('key3', 'value3', 3)
Success!

Executing: INSERT INTO ns2.table2 (key, value, version)
       VALUES
       ('keyA', 'valueA', 10),
       ('keyB', 'valueB', 20),
       ('keyC', 'valueC', 30)
Success!

Data in ns1.table1:
+-------+------------+-------+
|    key|       value|version|
+-------+------------+-------+
|   key1|      value1|      1|
|   key2|      value2|      2|
|   ke

25/08/14 17:21:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [16]:
spark = SparkSession.builder \
    .appName("Hive Federation Demo") \
    .config("spark.jars.packages", iceberg_package) \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.spark_catalog.uri", "thrift://localhost:9083") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "/tmp/data/spark-warehouse") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

# --- Verification ---
# Print the Spark version and some key configurations to verify.
print(f"Spark Session created successfully with Spark version: {spark.version}")
conf = spark.sparkContext.getConf()
print(f"Spark Configuration: {conf.getAll()}")

# Verify data in tables
print("\nData in ns1.table1:")
spark.sql("SELECT * FROM ns1.table1").show()

print("\nData in ns2.table2:")
spark.sql("SELECT * FROM ns2.table2").show()




25/08/14 17:21:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Spark Session created successfully with Spark version: 3.5.5
Spark Configuration: [('spark.files', 'file:///Users/pnilangekar/.ivy2/jars/org.apache.iceberg_iceberg-spark-runtime-3.5_2.12-1.9.0.jar'), ('spark.jars', 'file:///Users/pnilangekar/.ivy2/jars/org.apache.iceberg_iceberg-spark-runtime-3.5_2.12-1.9.0.jar'), ('spark.sql.variable.substitute', 'true'), ('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkCatalog'), ('spark.sql.catalog.hadoop_cat.warehouse', 'file:///tmp/warehouse'), ('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation', 'vended-credentials'), ('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.iceberg:iceberg-aws-bundle:1.7.1,software.amazon.awssdk:bundle:2.30.25,net.snowflake:snowflake-jdbc:3.13.32,com.azure:azure-storage-file-datalake:12.17.0,software.amazon.s3tables:s3-tables-catalog-for-iceberg-runtime:0.1.5'), ('spark.sql.catalog.polaris.type', 'rest'), ('spark.sql.catalog.polaris.token', 'eyJhbGc